<a href="https://colab.research.google.com/github/galihpermana292/smallest-value-cuda/blob/main/Kelompok_9_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-8ztnd3v0
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-8ztnd3v0
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=863f824a894b11a09eb58be33a8a0e7a5c1073cddaec9b08b46503782b0f25eb
  Stored in directory: /tmp/pip-ephem-wheel-cache-r376eks5/wheels/db/c1/1f/a2bb07bbb4a1ce3c43921252aeafaa6205f08637e292496f04
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include<stdio.h>
#include <iostream>
#include <cstdio>

using namespace std;

  // kerne findSmallest yang digunakan untuk mencari nilai terkecil
__global__ void findSmallest(int* input, int* sized, int* min_num){
  *min_num = *input;
  // proses mencari nilai terkecil dengan membandingan value setiap array
  for(int i = 1; i < *sized; i++){
    // jika menemukan value yang lebih kecil, simpan valuenya di variable min_num
    if(*input < *min_num){
      *min_num = *input;
    }
  }
}

int main( void ) {
  // mendapatkan size tipe data integer untuk dimasukkan ke variable size
  int size = sizeof( int );

  //host
  // value array yang mempunyai nilai yang nantinya dicari nilai terkecilnya
  int var_arr[] = {11, 22, 33, 44, 55, 66, 77};
  // ukuran arraynya
  int sizeArr = 7;
  // variable menampung nilai terkecilnya
  int findMin;

  //device
  // membuat pointer untuk ketiga variable yang ada di host
  int *deviceArr, *deviceSize, *deviceFindMinp;

  // alokasi memori pada setiap pointer device menggunakan size of integer
  cudaMalloc( (void**)&deviceArr, size );
  cudaMalloc( (void**)&deviceSize, size );
  cudaMalloc( (void**)&deviceFindMinp, size );

  // variable host yang memiliki value, kita copy ke device yaitu value array dan size array
  cudaMemcpy( deviceArr, &var_arr, size, cudaMemcpyHostToDevice );
  cudaMemcpy( deviceSize, &sizeArr, size, cudaMemcpyHostToDevice );
  
  // binding function findSmallest dengan thread dan block 1
  findSmallest<<< 1, 1 >>>( deviceArr, deviceSize, deviceFindMinp );

  // hasilnya kita copy balik ke host dari variable deviceFindMinp ke findMin
  cudaMemcpy( &findMin, deviceFindMinp, size, cudaMemcpyDeviceToHost );

  // output
  cout<< "nilai terkecil adalah: " << findMin << endl;

  // free alokasi memorinya
  cudaFree( deviceArr );
  cudaFree( deviceSize );
  cudaFree( deviceFindMinp );
  return 0;
}